# Fly Cell Atlas - HEAD - AUCell Oxphos scoring
*This script is to be run before the Rmd script, for computing the AUC values with pySCENIC* 

**Author:** Vincent Gardeux

**Date Created:** 13/02/2025
**Date Last Modified:** 13/02/2025

In [1]:
# import dependencies
import pandas as pd
import numpy as np
import h5py
from ctxcore.genesig import GeneSignature
from pyscenic.aucell import create_rankings, enrichment

In [2]:
GENE_SIGNATURE_FNAME = '/data/gardeux/Neuro_Droso_ND75KD/data/Oxphos_genes.xlsx'
GENE_SIGNATURE_SNAME = '68 OXPHOS genes'
EXPRESSION_MTX_FNAME = '/data/gardeux/Neuro_Droso_ND75KD/data/s_fca_biohub_head_10x.loom' # From https://cloud.flycellatlas.org/index.php/s/jtH5fGwLqtiSRSD

In [3]:
data_excel = pd.read_excel(io = GENE_SIGNATURE_FNAME, sheet_name = GENE_SIGNATURE_SNAME)
data_excel

,Complex,Complex_Name,Complex_Subunit,Gene_Name,Gene_CG,Gene_Ensembl_ID,Gene_Symbol,Gene_Symbol_clean
0,I,NADH ubiquinone oxidoreductase,34,13 kDa A,CG8680,FBgn0031684,ND-13A,ND-13A
1,I,NADH ubiquinone oxidoreductase,34,13 kDa B,CG6463,FBgn0047038,ND-13B,ND-13B
2,I,NADH ubiquinone oxidoreductase,34,15 kDa,CG11455,FBgn0031228,ND-15,ND-15
3,I,NADH ubiquinone oxidoreductase,34,18 kDa,CG12203,FBgn0031021,ND-18,ND-18
4,I,NADH ubiquinone oxidoreductase,34,19 kDa,CG3683,FBgn0035046,ND-19,ND-19
...,...,...,...,...,...,...,...,...
63,V,F0/F1 ATP synthase,13,F,CG4692,FBgn0035032,ATPsynF,ATPsynF
64,V,F0/F1 ATP synthase,13,G,CG6105,FBgn0010612,ATPsynG,ATPsynG
65,V,F0/F1 ATP synthase,13,Coupling factor 6,CG4412,FBgn0016119,ATPsynCF6,ATPsynCF6
66,V,F0/F1 ATP synthase,13,Lipid-binding protein,CG1746,FBgn0039830,ATPsynC,ATPsynC


In [4]:
#gs = GeneSignature('Flybase - GO - Oxidative phosphorylation', data_excel["Gene_Ensembl_ID"].tolist())
gs = GeneSignature('Flybase - GO - Oxidative phosphorylation', data_excel["Gene_Ensembl_ID"].tolist())
gs

GeneSignature(name='Flybase - GO - Oxidative phosphorylation', gene2weight=frozendict.frozendict({'FBgn0031684': 1.0, 'FBgn0047038': 1.0, 'FBgn0031228': 1.0, 'FBgn0031021': 1.0, 'FBgn0035046': 1.0, 'FBgn0030718': 1.0, 'FBgn0017567': 1.0, 'FBgn0030853': 1.0, 'FBgn0266582': 1.0, 'FBgn0037001': 1.0, 'FBgn0019957': 1.0, 'FBgn0039909': 1.0, 'FBgn0031771': 1.0, 'FBgn0017566': 1.0, 'FBgn0040705': 1.0, 'FBgn0034645': 1.0, 'FBgn0033570': 1.0, 'FBgn0025839': 1.0, 'FBgn0031505': 1.0, 'FBgn0034576': 1.0, 'FBgn0033961': 1.0, 'FBgn0029868': 1.0, 'FBgn0001989': 1.0, 'FBgn0031436': 1.0, 'FBgn0030605': 1.0, 'FBgn0032511': 1.0, 'FBgn0011361': 1.0, 'FBgn0029888': 1.0, 'FBgn0052230': 1.0, 'FBgn0029971': 1.0, 'FBgn0021967': 1.0, 'FBgn0011455': 1.0, 'FBgn0058002': 1.0, 'FBgn0085468': 1.0, 'FBgn0261439': 1.0, 'FBgn0014028': 1.0, 'FBgn0037873': 1.0, 'FBgn0039112': 1.0, 'FBgn0034245': 1.0, 'FBgn0011227': 1.0, 'FBgn0260008': 1.0, 'FBgn0030733': 1.0, 'FBgn0021906': 1.0, 'FBgn0035600': 1.0, 'FBgn0038271': 1.0, 'F

In [5]:
# Open Loom file in reading mode
f = h5py.File(EXPRESSION_MTX_FNAME, 'r')
m = f["/matrix"][:,:]
gene_names = f["/row_attrs/Gene"].asstr()[:]
cell_names = f["/col_attrs/CellID"].asstr()[:]
f.close()

m

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 7.,  1.,  1., ...,  2.,  3.,  1.],
       [19.,  2.,  4., ...,  1.,  2.,  1.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0.,  2.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [6]:
m.shape

(13056, 100527)

In [7]:
cell_names

array(['AAACCCAGTTACGGAG-d541ae4e__FCA1_MaleFemale_Head',
       'AAACCCATCCGTACGG-d541ae4e__FCA1_MaleFemale_Head',
       'AAACGAAGTATGATCC-d541ae4e__FCA1_MaleFemale_Head', ...,
       'TTTGTTGTCATCGCCT-f7ed0992__FCA15_Female_head_adult',
       'TTTGTTGTCCATTCGC-f7ed0992__FCA15_Female_head_adult',
       'TTTGTTGTCGCTTGCT-f7ed0992__FCA15_Female_head_adult'], dtype=object)

In [8]:
len(cell_names)

100527

In [9]:
gene_names

array(['128up', '14-3-3epsilon', '14-3-3zeta', ..., 'zormin', 'zyd',
       'zye'], dtype=object)

In [10]:
len(gene_names)

13056

In [11]:
# [Input] Fixed gene annotation (for SCENIC feather file compatibility)
genome_data = pd.read_csv("/data/gardeux/Neuro_Droso_ND75KD/data/dmel-all-r6.31-premrna.gene_mapping.txt", sep="\t", na_filter=False)

# Update gene names using fixing table
gene_to_ensembl_dict = genome_data.set_index('gene_name')['gene_id'].to_dict()

# Check if all gene names are in the dictionary
missing_genes = [gene for gene in gene_names if gene not in gene_to_ensembl_dict]
if missing_genes:
    print(f"Error: The following genes are missing from the annotation table: {len(missing_genes)}")
else:
    # Fix gene names
    gene_names_updated = np.array([
        gene_to_ensembl_dict[gene] for gene in gene_names
    ])

    print(gene_names_updated)

['FBgn0010339' 'FBgn0020238' 'FBgn0004907' ... 'FBgn0052311' 'FBgn0265767'
 'FBgn0036985']


In [12]:
len(gene_names_updated)

13056

In [13]:
ex_matrix = pd.DataFrame(m, columns = cell_names, index = gene_names_updated)
ex_matrix

,AAACCCAGTTACGGAG-d541ae4e__FCA1_MaleFemale_Head,AAACCCATCCGTACGG-d541ae4e__FCA1_MaleFemale_Head,AAACGAAGTATGATCC-d541ae4e__FCA1_MaleFemale_Head,AAACGAATCACCTACC-d541ae4e__FCA1_MaleFemale_Head,AAACGCTAGAAACCAT-d541ae4e__FCA1_MaleFemale_Head,AAACGCTAGCAATAGT-d541ae4e__FCA1_MaleFemale_Head,AAACGCTCAACTTCTT-d541ae4e__FCA1_MaleFemale_Head,AAACGCTTCAGACCTA-d541ae4e__FCA1_MaleFemale_Head,AAAGAACAGCAACTCT-d541ae4e__FCA1_MaleFemale_Head,AAAGAACCACTTGTGA-d541ae4e__FCA1_MaleFemale_Head,...,TTTGGTTAGTAATTGG-f7ed0992__FCA15_Female_head_adult,TTTGGTTAGTCCTACA-f7ed0992__FCA15_Female_head_adult,TTTGGTTCACATACTG-f7ed0992__FCA15_Female_head_adult,TTTGGTTCATCGGTTA-f7ed0992__FCA15_Female_head_adult,TTTGGTTCATTGACCA-f7ed0992__FCA15_Female_head_adult,TTTGGTTGTACGAAAT-f7ed0992__FCA15_Female_head_adult,TTTGTTGCAACGCATT-f7ed0992__FCA15_Female_head_adult,TTTGTTGTCATCGCCT-f7ed0992__FCA15_Female_head_adult,TTTGTTGTCCATTCGC-f7ed0992__FCA15_Female_head_adult,TTTGTTGTCGCTTGCT-f7ed0992__FCA15_Female_head_adult
FBgn0010339,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FBgn0020238,7.0,1.0,1.0,0.0,1.0,0.0,5.0,2.0,1.0,4.0,...,2.0,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,1.0
FBgn0004907,19.0,2.0,4.0,2.0,3.0,3.0,7.0,1.0,2.0,3.0,...,2.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,2.0,1.0
FBgn0010340,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FBgn0085813,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FBgn0265434,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
FBgn0259789,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FBgn0052311,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,2.0
FBgn0265767,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0


In [14]:
# Compute the intersection
intersection = list(set(gene_names).intersection(set(data_excel["Gene_Ensembl_ID"].tolist())))
print(intersection)

[]


In [15]:
len(intersection)

0

In [ ]:
# Run AUCell
# 1. Rankings (~2mn)
rnk_mtx = create_rankings(ex_matrix.transpose(), seed=42)

In [ ]:
# 2. Enrichment (~5s)
aucs = enrichment(rnk_mtx, gs)
aucs

In [ ]:
min(aucs["AUC"])

In [ ]:
max(aucs["AUC"])

In [ ]:
# Save DataFrame to TSV file
aucs.to_csv("./data/Pan_neuro_integrated_68_Oxphos_AUCell_auc.tsv", sep='\t', index=True)